### Observations


Following are the observations from the different datasets:

#### Users Dataset:

1. Documentation says-  users.`role: constant value set to 'CONSUMER'`. But 2 unique values found in dataset.
2. SignUpSource, State and lastLogin has nan values.
3. The users table should only have unique user ids to avoid redundancy of data and efficient querying.
4. All dates are a numeric value. Unknown if dates were lost in conversion or dummy data.


#### Brands Dataset:

1. Brands dataset has no uniquely identifying attribute.
2. `brandCode` has missing/nan values.

#### Reciepts Dataset:

1. Some queries in Part 2 are based on `Status = 'Accepted'`. There is no such data in the dataset. Might be an data anomaly.
2. Multiple date columns. Based on what they represent and business cases some can may be dropped to reduce redundancy. <br>
e.g. `createDate`: The date that the event was created <br>
     `purchaseDate`: the date of the purchase<br>
In real world, most likely these should be the same date.

3. `recieptItemList` has non-uniform dictionaries. <br>
    a. `Item Number` which could be potential identifier for each recieptlistitem is sparsely populated. <br>
    b. Some have `barcode`, `finalPrice` while some don't making it difficult to check data integrity among sum of individual items in recieptItemList and compare against totalSpent in reciept.This data issue needs to be first taken care of before futher enhancing the user defined function to conduct data integrity checks.Similar checks can be designed for item quantity at receipt and receipt item levels too. <br>
    c. There are multiple price columns which mostly has same data e.g.`discountedItemPrice`, `finalPrice`, `itemPrice`, `targetPrice`

---

### Code used to analyse datasets

#### Imports

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

#### Load Data

In [2]:
users = pd.read_json('./data/users.json/users.json', lines=True)
brands = pd.read_json('./data/brands.json/brands.json', lines=True)
reciepts = pd.read_json('./data/receipts.json/receipts.json', lines=True)

#### Users Dataset Analysis

In [3]:
#Explore users dataset on  a high level
users.head(5)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [4]:
users.dtypes  

_id             object
active            bool
createdDate     object
lastLogin       object
role            object
signUpSource    object
state           object
dtype: object

In [5]:
users.shape

(495, 7)

In [6]:
non_dic_columns=['active','role','signUpSource','state']

print("Unique values in dict columns:")
for col in non_dic_columns:
    print(col,'---', users[col].unique())

Unique values in dict columns:
active --- [ True False]
role --- ['consumer' 'fetch-staff']
signUpSource --- ['Email' 'Google' nan]
state --- ['WI' 'KY' 'AL' 'CO' 'IL' nan 'OH' 'SC' 'NH']


In [7]:
def convert_to_string(val):
    return str(val)

users['id'] = users['_id'].apply(convert_to_string)

print('Unique user ids:', users.id.nunique())
print('Total user ids:', len(users.id))


Unique user ids: 212
Total user ids: 495


#### Structured Data of Users

In [8]:
df_users = pd.read_excel('./data/users.json/users.xlsx')
df_users

,id,stateID,isActive,dateLastLogin,dateCreated,signUpSource
0,5ff1e194b6a9d73a3a9f1052,WI,True,1.609688e+12,1609687444800,Email
1,5ff1e194b6a9d73a3a9f1052,WI,True,1.609688e+12,1609687444800,Email
2,5ff1e194b6a9d73a3a9f1052,WI,True,1.609688e+12,1609687444800,Email
3,5ff1e1eacfcf6c399c274ae6,WI,True,1.609688e+12,1609687530554,Email
4,5ff1e194b6a9d73a3a9f1052,WI,True,1.609688e+12,1609687444800,Email
...,...,...,...,...,...,...
490,54943462e4b07e684157a532,NaN,True,1.614963e+12,1418998882381,NaN
491,54943462e4b07e684157a532,NaN,True,1.614963e+12,1418998882381,NaN
492,54943462e4b07e684157a532,NaN,True,1.614963e+12,1418998882381,NaN
493,54943462e4b07e684157a532,NaN,True,1.614963e+12,1418998882381,NaN


In [9]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             495 non-null    object 
 1   stateID        436 non-null    object 
 2   isActive       495 non-null    bool   
 3   dateLastLogin  433 non-null    float64
 4   dateCreated    495 non-null    int64  
 5   signUpSource   388 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(3)
memory usage: 19.9+ KB


In [22]:
for col in df_users.columns:
    print('Unique count of',col,'---', df_users[col].nunique())

Unique count of id --- 212
Unique count of stateID --- 7
Unique count of isActive --- 2
Unique count of dateLastLogin --- 172
Unique count of dateCreated --- 212
Unique count of signUpSource --- 4


#### Brands Dataset Analysis

In [31]:
brands.head(5)
brands.shape

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


(1167, 8)

In [26]:
brands.dtypes

_id              object
barcode           int64
category         object
categoryCode     object
cpg              object
name             object
topBrand        float64
brandCode        object
dtype: object

In [33]:
print('Unique count of Brands cols:')

brands[['barcode','name','brandCode']].nunique()

Unique count of Brands cols:


barcode      1160
name         1156
brandCode     897
dtype: int64

In [42]:
for col in ['barcode','name','brandCode']:
    print('Unique values of',col,'--- \n', brands[col].unique())

Unique values of barcode --- 
 [511111019862 511111519928 511111819905 ... 511111416173 511111400608
 511111019930]
Unique values of name --- 
 ['test brand @1612366101024' 'Starbucks' 'test brand @1612366146176' ...
 'test brand @1598639215217' 'LIPTON TEA Leaves'
 'test brand @1613158231643']
Unique values of brandCode --- 
 [nan 'STARBUCKS' 'TEST BRANDCODE @1612366146176'
 'TEST BRANDCODE @1612366146051' 'TEST BRANDCODE @1612366146827'
 'TEST BRANDCODE @1612366146091' 'TEST BRANDCODE @1612366146133'
 'J.L. KRAFT' 'CAMPBELLS HOME STYLE' 'TEST'
 'TEST BRANDCODE @1598813526777' 'CALUMET' '511111205012'
 'AUNT JEMIMA SYRUP' 'MOLSON' 'LOTRIMIN' 'TEST BRANDCODE @1597342520277'
 'ST IVES' 'CHRISIMAGE' 'ALKA SELTZER' "JACK DANIEL'S BARBECUE"
 'MAGNUM Ice Cream' '511111105329' 'TEST BRANDCODE @1598635634882'
 'TACO BELL' 'FROSTED CHEERIOS' 'TEST BRANDCODE @1598639199674'
 'GODIVA DRY PACKAGED DESSERTS' 'LARABAR' 'TEST BRANDCODE @1597350074333'
 'TEST BRANDCODE @1607636368717' 'TEST BRANDCODE

#### Reciepts Dataset Analysis

In [11]:
#Explore reciepts dataset on  a high level
reciepts.head(5)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [12]:
reciepts.dtypes  

_id                         object
bonusPointsEarned          float64
bonusPointsEarnedReason     object
createDate                  object
dateScanned                 object
finishedDate                object
modifyDate                  object
pointsAwardedDate           object
pointsEarned               float64
purchaseDate                object
purchasedItemCount         float64
rewardsReceiptItemList      object
rewardsReceiptStatus        object
totalSpent                 float64
userId                      object
dtype: object

In [13]:
reciepts.shape

(1119, 15)

In [14]:
reciepts.rewardsReceiptStatus.unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

In [15]:
reciepts.bonusPointsEarnedReason.value_counts()

All-receipts receipt bonus                                                             183
Receipt number 1 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)    119
COMPLETE_NONPARTNER_RECEIPT                                                             71
COMPLETE_PARTNER_RECEIPT                                                                39
Receipt number 3 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)     31
Receipt number 2 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)     30
Receipt number 5 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)     27
Receipt number 4 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)     26
Receipt number 6 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36)     18
Name: bonusPointsEarnedReason, dtype: int64

In [16]:
df_temp =reciepts[['bonusPointsEarnedReason','bonusPointsEarned','pointsEarned']]
df_temp = df_temp.drop_duplicates()
df_temp[df_temp['bonusPointsEarned'] == df_temp['pointsEarned']]

,bonusPointsEarnedReason,bonusPointsEarned,pointsEarned
0,"Receipt number 2 completed, bonus point schedu...",500.0,500.0
1,"Receipt number 5 completed, bonus point schedu...",150.0,150.0
2,All-receipts receipt bonus,5.0,5.0
5,"Receipt number 1 completed, bonus point schedu...",750.0,750.0
9,"Receipt number 3 completed, bonus point schedu...",250.0,250.0
10,"Receipt number 6 completed, bonus point schedu...",100.0,100.0
14,"Receipt number 4 completed, bonus point schedu...",300.0,300.0
47,COMPLETE_NONPARTNER_RECEIPT,25.0,25.0


In [17]:
df_temp[df_temp['bonusPointsEarned'] != df_temp['pointsEarned']]

,bonusPointsEarnedReason,bonusPointsEarned,pointsEarned
12,"Receipt number 5 completed, bonus point schedu...",150.0,8850.0
15,NaN,NaN,NaN
20,"Receipt number 4 completed, bonus point schedu...",300.0,389.2
23,COMPLETE_PARTNER_RECEIPT,40.0,185.0
27,COMPLETE_NONPARTNER_RECEIPT,25.0,35.0
...,...,...,...
643,NaN,NaN,209.8
664,NaN,NaN,210.0
667,NaN,NaN,209.5
917,"Receipt number 1 completed, bonus point schedu...",750.0,789.2


##### Comments: 

A clear understanding of how the data is saved to `bonusPointsEarned` and `pointsEarned` cannot be gained. Sometimes the value is same and sometimes it is not. There could possibly be a better way to store this data.

In [18]:
reciepts.rewardsReceiptItemList

0       [{'barcode': '4011', 'description': 'ITEM NOT ...
1       [{'barcode': '4011', 'description': 'ITEM NOT ...
2       [{'needsFetchReview': False, 'partnerItemId': ...
3       [{'barcode': '4011', 'description': 'ITEM NOT ...
4       [{'barcode': '4011', 'description': 'ITEM NOT ...
                              ...                        
1114    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1115                                                  NaN
1116                                                  NaN
1117    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1118                                                  NaN
Name: rewardsReceiptItemList, Length: 1119, dtype: object

##### Comments :

reciepts violates 1NF due to presence of nested data/array. Hence it is not a relational model.
Relation Decomposition is necessary to convert it into a relational model.

##### Test data integrity of data

In [19]:
#udf for aggregating total finalPrice of each item in reciept item list of a given reciept

def agg_finalPrice_receipt_itms(row):
    columns = ['barcode','description','finalPrice','quantityPurchased']
    df_rct_itms = pd.DataFrame(columns = columns)
    data = []

    for dic in row['rewardsReceiptItemList']:
#             for dic in item:
                values =[dic.get('barcode')
                        ,dic.get('description')
                        ,float(dic.get('finalPrice'))
                        ,dic.get('quantityPurchased')]
                zipped = zip(columns, values)
                a_dictionary = dict(zipped)
                data.append(a_dictionary)

    df_rct_itms = df_rct_itms.append(data, True)
    return [row.totalSpent,round(df_rct_itms.finalPrice.sum(),2)]

In [20]:
reciepts.iloc[[423]].apply(agg_finalPrice_receipt_itms, axis=1)
reciepts.iloc[[650]].apply(agg_finalPrice_receipt_itms, axis=1)

423    [1183.1, 1183.1]
dtype: object

650    [28.0, 28.0]
dtype: object

##### Noteworthy:
This code could be further enhanced to make a mapping of reciept index, totalSpent, sum(finalPriceofRctItems) on entire dataset

In [21]:
reciepts.apply(agg_finalPrice_receipt_itms, axis=1)

TypeError: float() argument must be a string or a number, not 'NoneType'

In [ ]:
for item in reciepts.iloc[[423]].rewardsReceiptItemList:
     for dic in item:
        print([k  for  k in  dic.keys()])